In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.auto import tqdm
import time
import numpy as np
import pandas as pd

data_path = '.'



In [ ]:
kor_trans_list

In [25]:
data_kor_to_eng_pd = pd.read_csv('back_translation_result.csv')
data_kor_to_eng_pd

,Unnamed: 0,kor_to_en,en_to_kor,origin_text
0,0,"""Something"" is a song written by George Harris...","""Something""은 조지 해리슨이 작곡하고 비틀즈의 1969년 앨범 ""Abbey...",〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...
1,1,"The Bareunmirae Party, the New Alternative Par...","호남을 기반으로 한 미래당과 대안신당, 민주평화당이 우여곡절 끝에 민생당(가칭)으로...",호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...
2,2,"Gwangju FC, which ranks first in the K League ...",K리그2 1위 광주FC가 26일 한국프로축구연맹의 관중 유치와 마케팅 성과를 인정받...,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...
3,3,"Asung Daiso Co., Ltd., a uniform price househo...",균일가 생활용품 판매점 아성다이소(주)가 COVID-19 바이러스로 몸살을 앓고 있...,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...


In [3]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver
driver=chrome_setting()

In [4]:
trans_list = []

In [67]:
from load_data import *
df = load_data('../dataset/train/train.csv')
df['sentence']

0        〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...
1        호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...
2        K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...
3        균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...
4        1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...
                               ...                        
32465    한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...
32466    법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...
32467    완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...
32468    중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...
32469    화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...
Name: sentence, Length: 32470, dtype: object

In [77]:
from urllib.request import urlopen
import json
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      query = 'https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st="' + text_data[i] + '"'
      print(query)
      driver.get(query)
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(1.5)
      backtrans = element.text 
      print(f'element text{element.text}')

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except BaseException as e:
      print(e)
      page = urlopen(query)
      # bytes to string
      doc = page.read().decode('utf-8')
      # string to dictionary
      dic = json.loads(doc)
      result_dict = dic['result']['trades']
      trans_list.append('')

In [78]:
stop_words = ['〈', '〉', '(', ')', '<', '>', "《", '》']
def remove_stop_words(s):
    for w in stop_words:
        s = s.replace(w, '')
        ''.join(s)
    return s

In [79]:
trans_list = []
test_sentence = [remove_stop_words(df.loc[0, 'sentence'])]
print(test_sentence)
kor_to_trans(test_sentence, 'en', 0, 1)

['Something는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 Abbey Road에 담은 노래다.']


  0%|          | 0/1 [00:00<?, ?it/s]

https://papago.naver.com/?sk=ko&tk=en&st="Something는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 Abbey Road에 담은 노래다."
HTTPConnectionPool(host='127.0.0.1', port=43698): Max retries exceeded with url: /session/826e451add93e99cc37184cf9adbb3df/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f819ac75e80>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [70]:
print(trans_list)
print(' '.join(trans_list))

['']



In [12]:
kor_trans_list = np.load('./kor_to_eng_final.npy')
print(kor_trans_list.size)


en_trans_list = np.load('./en_to_kor_final.npy')
print(en_trans_list.size)
en_trans_list[:2]

31903
31890


array(['"Something"은 조지 해리슨이 작곡하고 비틀즈의 1969년 앨범 "Abbey Road"에 수록된 곡이다.',
       '호남을 기반으로 한 미래당과 대안신당, 민주평화당이 우여곡절 끝에 민생당(가칭)으로 거듭난다.'],
      dtype='<U391')

In [14]:
with open('en_to_kor_final.txt', 'w', encoding='utf-8-sig') as f:
    for s in en_trans_list:
        f.write(f'{s}\n')